### Exercises

#### Question 1

Write some code that generates a file containing containing rows containing the following data:

```
i, fibonacci_i, factorial_i, gcd_fibonacci_i_factorial_i
```

where:
- `i`: integer values from `0` to `100`
- `fibonacci_i`: the `i`th Fibonacci number
- `factorial_i`: the factorial of `i` (`i!`)
- `gcd_fib_i_fact_i`: the greatest common denominator of the `i`th Fibonacci number and `i!` 

Hint: look at the `math.factorial` and `math.gcd` functions in the Python docs

Also make sure to include a header row in your file.

For example, the first few rows in your file should contain this data:

```
i,fib,fact,gcd
0,1,1,1
1,1,1,1
2,2,2,2
3,3,6,3
4,5,24,1
5,8,120,8
```

##### Solution 1

In [276]:
import math
from functools import lru_cache
from timeit import timeit

In [104]:
file = 'question_1_solution.csv'

In [80]:
def fib_fac_gcd_output(n_th):
    i = range(n_th+1)
    
    @lru_cache(maxsize=3)
    def fib(n):
        if n <= 1:
            return n
        return fib(n-1) + fib(n-2)
    
    num_series = (
        (num, fib(num), math.factorial(num), math.gcd(fib(num), math.factorial(num))) for num in i
    )

    output = (
        ','.join([str(num), str(fib_i), str(fac_i), str(gcd_fib_fac)]) + '\n' 
        for num, fib_i, fac_i, gcd_fib_fac in num_series
    )

    return output

In [83]:
list(fib_fac_gcd_output(5))

['0,0,1,1\n',
 '1,1,1,1\n',
 '2,1,2,1\n',
 '3,2,6,2\n',
 '4,3,24,3\n',
 '5,5,120,5\n']

In [106]:
with open(file, 'w') as f:
    f.write('i,fib,fact,gcd\n')
    for line in fib_fac_gcd_output(100):
        f.write(line)

In [110]:
with open(file) as f:
    for _ in range(12):
        print(next(f), end='')

i,fib,fact,gcd
0,0,1,1
1,1,1,1
2,1,2,1
3,2,6,2
4,3,24,3
5,5,120,5
6,8,720,8
7,13,5040,1
8,21,40320,21
9,34,362880,2
10,55,3628800,5


#### Question 2

Using the file you just generated, write three functions:
- `fib`
- `fact`
- `gcd_fib_fact`

that perform the same calculations as our original `fib` function, the `math` module's `factorial` and the `gcd` of the corresponding fibonacci and factorial numbers, but uses the data that was saved in the file as a cache/lookup mechanism - i.e. just use the numbers in the file if they are available, otherwise make the calculation.

##### Solution 2

In [113]:
from pprint import pprint

In [190]:
cache = {}

with open(file) as f:
    header = next(f)

    for data in f:
        num, fib, fac, gcd = data.strip().split(',')
        cache[int(num)] = {
            'fib': int(fib),
            'fact': int(fac),
            'gcd': int(gcd)
        }

In [263]:
@lru_cache(maxsize=3)
def new_fib(n):
    if n in cache:
        #print('cache hit')
        return cache[n]['fib']
    else:
        #print('cache miss')
        if n <= 1:
            return n
        return new_fib(n-1) + new_fib(n-2)

def new_fact(n):
    if n in cache:
        #print('cache hit')
        return cache[n]['fact']
    else:
        #print('cache miss')
        return math.factorial(n)

def new_gcd(*args, n):
    if n in cache:
        #print('cache hit')
        return cache[n]['gcd']
    else:
        #print('cache miss')
        return math.gcd(*args)

In [219]:
new_fib(92)

cache hit


7540113804746346429

In [196]:
new_fact(100)

cache hit


93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

In [224]:
i = 84
new_gcd(new_fib(i), new_fact(i), n=i)

cache hit


4505904

In [226]:
def ffg_from_cache(n_th):
    i = range(n_th+1)
    
    num_series = (
        (num, new_fib(num), new_fact(num), new_gcd(new_fib(num), new_fact(num), n=num)) for num in i
    )

    output = (
        ','.join([str(num), str(fib_i), str(fac_i), str(gcd_fib_fac)]) + '\n' 
        for num, fib_i, fac_i, gcd_fib_fac in num_series
    )

    return output

In [274]:
list(ffg_from_cache(10))

['0,0,1,1\n',
 '1,1,1,1\n',
 '2,1,2,1\n',
 '3,2,6,2\n',
 '4,3,24,3\n',
 '5,5,120,5\n',
 '6,8,720,8\n',
 '7,13,5040,1\n',
 '8,21,40320,21\n',
 '9,34,362880,2\n',
 '10,55,3628800,5\n']

In [273]:
timeit('list(ffg_from_cache(10))', globals=globals(), number=1)

5.9200014220550656e-05

In [275]:
timeit('list(fib_fac_gcd_output(10))', globals=globals(), number=1)

0.0002834000042639673